# Efficient data analysis with pandas

In [ ]:
import pandas as pd

# read csv files
weather_entries = pd.read_csv("weather_station_information.csv", sep='\t')
stations = pd.read_csv("stations.csv", sep=',')
weather_entries

In [ ]:
stations

### column operations

In [ ]:
# column aggregations
total_rain_water = weather_entries['precipitation'].sum()
avg_height_weather_station = stations['HGHT'].mean()
max_temp_ever = weather_entries['max_temp'].max()
count_weather_station = stations['STAID'].count()
number_of_countries = stations['CN'].nunique()
print "total rain water collected by weather stations: %i" % (total_rain_water,)
print "average height of weather station: %i" % (avg_height_weather_station,)
print "highest temperature recorded since 2010: %i degrees" % (max_temp_ever,)
print "count of weather stations: %i" % (count_weather_station,)
print "number of countries members: %i" % (number_of_countries,)

In [ ]:
# operations between columns 
weather_entries['daily_diff'] = weather_entries['max_temp'] - weather_entries['min_temp']
weather_entries

In [ ]:
weather_entries.describe()

In [ ]:
# special function
stations['CN'].value_counts()
# sort by value

In [ ]:
# return normalize values
stations['CN'].value_counts(normalize=True)

In [ ]:
# parse dates
# create column with month and year
weather_entries['date'] = pd.to_datetime(weather_entries['date'], format="%Y%m%d")
weather_entries['month'] = weather_entries.date.dt.month
weather_entries['year'] = weather_entries.date.dt.year
weather_entries

In [ ]:
# stations in berlin
stations[stations.STANAME.str.contains("BERLIN")]

In [ ]:
# advance filtering
berlin_stations = stations[stations.STANAME.str.contains("BERLIN-")].STAID
print "Weather_stations in Berlin: %i" % (len(berlin_stations))
berlin_entries = weather_entries[weather_entries.station.isin(berlin_stations)]
berlin_entries

"~" operator perform not in

### Divide and conquer: groupby

In [ ]:
# groupby on multiple dimensions
month_agg = berlin_entries.groupby(['station', 'month']).mean().round(0)
month_agg

In [ ]:
# aggregate on multiindex
# agg function
agg_dict = {'max_temp': ['max', 'min', 'mean', 'count'], 'min_temp': ['max', 'min', 'mean'], 'precipitation':['max', 'min', 'mean']}
station_agg = month_agg.groupby(level=0).agg(agg_dict).round(2)
station_agg

In [ ]:
# groupby as iterator is a good way to slide the data
for station, df in berlin_entries.groupby('station'):
    print station
    print "*" * 50
    print df.head(1)
    print "=" * 50

# Understanding the data: Plotting

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

berlin_entries.station.value_counts().plot(kind='bar')

In [ ]:
berlin_entries.groupby('month').station.count().plot(ylim=[0,3100])

For comparison always set the y-axis to 0 to observe the total impact

Information for the last months of the year is lower because the data goes until 30.08.2016

In [ ]:
berlin_full_year = berlin_entries[berlin_entries.year < 2016]

In [ ]:
berlin_full_year.groupby('month').station.count().plot(ylim=[0,3100])

So what is good weather in Berlin????
What should we take as indicator ? max_temp / avg_temp / min_temp ?

In [ ]:
temp_dist = dict()
for col in ['max_temp', 'avg_temp', 'min_temp']:
    dist = berlin_full_year[col].value_counts(normalize=True)
    temp_dist[col] = dist
    dist.plot(label=col, legend=True, title='distribution plot')
plt.show()

Value_counts sort the data by value desc, plt does not sort columns.

the min_temp bell is thiner and taller vs the max_temp curve is shorter and more spread.

In [ ]:
for temp, dist in temp_dist.iteritems():
    dist.cumsum().plot(label=temp, legend=True, title='cummulative distribution plot')

In [ ]:
temp_dist['avg_temp'].cumsum()

In [ ]:
berlin_full_year.precipitation.value_counts(normalize=True).sort_index().cumsum().plot()

Temperature and precipitation follow different distribution (normal and exponential respectively)

In [ ]:
berlin_full_year['rainy_days'] = berlin_full_year.precipitation.apply(lambda x: 1 if x > 0 else 0)
agg_dict2 = {'avg_temp': 'mean', 'daily_diff': 'mean', 'rainy_days': 'sum'}
station_final = berlin_full_year.groupby(['station', 'year']).agg(agg_dict2).groupby(level=0).mean()
station_final

In [ ]:
plt.scatter(station_final.avg_temp, station_final.rainy_days, station_final.daily_diff)

In [ ]:
station_final[(station_final.rainy_days < 100) & (station_final.avg_temp > 9.8)].join(stations.set_index('STAID'))